In [ ]:
from OptimizedDataGenerator4 import *
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras_tuner as kt
noGPU=False
if noGPU:
    tf.config.set_visible_devices([], 'GPU')

print("\nHIIIIIIIIIIIIIIIIII\n")

print(tf.config.experimental.list_physical_devices())
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print(tf.test.is_gpu_available())


In [ ]:
#// plotting code from google ai https://www.google.com/search?client=firefox-b-1-d&q=plot+tensorflow+model+history
# then modified
def plotModelHistory(history,modelNum = -999):
    plt.subplot(211)
    # Plot training & validation loss values
    plt.plot(history.history['loss'],label="Train")
    plt.plot(history.history['val_loss'],label="Validation")
    plt.title(f'Model {modelNum} loss and accuracy')
    plt.ylabel('Loss')
    # plt.xlabel('Epoch')
    plt.legend()
    plt.subplot(212)
    # Plot training & validation accuracy values
    plt.plot(history.history['binary_accuracy'],label="Train")
    plt.plot(history.history['val_binary_accuracy'],label="Validation")
    # plt.title(f'Model {modelNum} accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()


In [ ]:
validation_dir = "./tf_records1000Daniel/tfrecords_validation/"
train_dir = "./tf_records1000Daniel/tfrecords_train/"
x_feature_description: list = ['x_size','z_global','y_profile','x_profile','cluster', 'y_size', 'x_size', 'y_local']
trainODG = OptimizedDataGenerator(tf_records_dir=train_dir,load_records=True, x_feature_description=x_feature_description)
validationODG = OptimizedDataGenerator(tf_records_dir=validation_dir,load_records=True, x_feature_description=x_feature_description)
# trainODG._parse_tfrecord_fn()
## where do we tell it to use the z-global

In [ ]:
## HYPERPARAMETER SEARCH
from qkeras import QDense, QActivation, quantized_bits, quantized_relu

def model_builder(hp):

    # ── B) Architecture hyperparams ──────────────────────────────────────────
    # separately tune rows and cols

    row1nodes      = hp.Int("1",   10, 200, step=10)
    row2nodes      = hp.Int("2",   10, 200, step=10)
    row3nodes      = hp.Int("3",   10, 200, step=10)
    row4nodes      = hp.Int("4",   10, 200, step=10)
    row5nodes      = hp.Int("5",   10, 200, step=10)



    input1 = tf.keras.layers.Input(shape=(1,), name="z_global")
    input2 = tf.keras.layers.Input(shape=(1,), name="x_size")
    input3 = tf.keras.layers.Input(shape=(1,), name="y_size")
    input4 = tf.keras.layers.Input(shape=(1,), name="y_local")

    ## concatenate the inputs into one layer
    inputList = [input1, input2, input3, input4]
    inputs = tf.keras.layers.Concatenate()(inputList)


    ## here i will add the layers 

    # layer 1
    x = tf.keras.layers.Dense(row1nodes,activation='relu')(inputs)
    x = tf.keras.layers.Dense(row2nodes, activation='relu')(x)
    x = tf.keras.layers.Dense(row3nodes, activation='relu')(x)
    x = tf.keras.layers.Dense(row4nodes, activation='relu')(x)
    x = tf.keras.layers.Dense(row5nodes, activation='relu')(x)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputList, outputs=output)

    model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],
    run_eagerly  = True 
    )
    return model
     

In [ ]:
tuner = kt.RandomSearch(
    model_builder,
    objective           = "val_binary_accuracy",
    max_trials          = 120,
    executions_per_trial = 2,
    project_name        = "new_hyperparam_search"
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

tuner.search(
    trainODG,
    validation_data = validationODG,
    epochs          = 110,
    callbacks       = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
)

In [ ]:
tuner.results_summary()